In [34]:
from selenium import webdriver
import pandas as pd
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.expected_conditions import presence_of_element_located
from selenium.webdriver.common.by import By
from datetime import datetime

In [35]:
nc_lic = pd.read_excel('../../Data/License/NC_FULL_FILE_NUM_ONLY_110520.xlsx')

In [16]:
nc_lic = nc_lic.dropna(subset=['LIC_NBR'])
nc_lic = nc_lic.drop_duplicates('LIC_NBR')
len(nc_lic)

68826

In [23]:
license_list = list(nc_lic['License_Number'])

In [4]:
def wait(browser):
    '''Define wait'''
    return WebDriverWait(browser, 30)

In [2]:
license_number = '2007-01286'

In [31]:
def get_license_info(license_number, driver):
    license_input = wait(driver).until(presence_of_element_located((By.ID, "txtLicNum")))
    license_input.clear()
    license_input.send_keys(license_number)
    submit = driver.find_element_by_id("btnSubmit")
    submit.click()
    table =  wait(driver).until(presence_of_element_located((By.TAG_NAME,'tbody')))
    info_link = table.find_elements_by_tag_name('a')
    if info_link:
        info_link[0].click()
        license_statuses = driver.find_elements_by_xpath('//*[@id="pnlVerificationBody"]/div[1]/div[1]')
        if license_statuses:
            license_type = license_statuses[0].text
        else:
            license_type = 'None'
    else:
        license_type = 'Not found'
    new_search = wait(driver).until(presence_of_element_located((By.XPATH,'//*[@id="content"]/div/div[2]/div[3]/a[3]')))
    new_search.click()
    
    return(license_type, driver)

In [17]:
driver = webdriver.Chrome()
driver.get('https://portal.ncmedboard.org/verification/search.aspx')



In [18]:
license_type

'MD Full and Unrestricted'

In [19]:
license_dict_list = []
index=0

In [33]:
driver = webdriver.Chrome()

driver.get('https://portal.ncmedboard.org/verification/search.aspx')
for license in license_list[index:]:
    license_type, driver = get_license_info(license, driver)
    license_dict = {
        'License_Number': license,
        'License_Type': license_type
    }
    print(license_dict)
    license_dict_list.append(license_dict)
    index+=1

{'License_Number': '2015-00792', 'License_Type': 'Antonia B Ahern - MD Full and Unrestricted'}
{'License_Number': '2020-03547', 'License_Type': 'Matthew Todd Ahern - DO Full and Unrestricted'}
{'License_Number': 37782, 'License_Type': 'Gerald Thomas Ahigian - MD Full and Unrestricted'}
{'License_Number': 200500147, 'License_Type': 'David Joseph Ahlberg - MD Full and Unrestricted'}
{'License_Number': '2013-01171', 'License_Type': 'David Andrew Ahlers - MD Full and Unrestricted'}


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=86.0.4240.111)
  (Driver info: chromedriver=2.42.591088 (7b2b2dca23cca0862f674758c9a3933e685c27d5),platform=Windows NT 10.0.18362 x86_64)


In [90]:
driver.quit()
TODAY = str(datetime.today()).replace(':','').replace('.','').replace(' ','_')
pd.DataFrame(license_dict_list).to_csv(F'../../Data/License/NC_License_Status_{TODAY}.csv', index=False)

In [27]:
license_dict_list

[{'License_Number': '2007-01286', 'License_Type': 'MD Full and Unrestricted'},
 {'License_Number': '2018-02560', 'License_Type': 'MD Full and Unrestricted'},
 {'License_Number': '2013-02286', 'License_Type': 'MD Full and Unrestricted'},
 {'License_Number': 38886, 'License_Type': 'MD Full and Unrestricted'},
 {'License_Number': 38647, 'License_Type': 'MD Full and Unrestricted'},
 {'License_Number': 200100205, 'License_Type': 'MD Full and Unrestricted'},
 {'License_Number': '2019-01910', 'License_Type': 'MD Full and Unrestricted'},
 {'License_Number': '2012-00426', 'License_Type': 'MD Full and Unrestricted'},
 {'License_Number': '2017-02506', 'License_Type': 'MD Full and Unrestricted'},
 {'License_Number': 9800751, 'License_Type': 'MD Full and Unrestricted'},
 {'License_Number': '2020-02397', 'License_Type': 'MD Full and Unrestricted'},
 {'License_Number': '2016-01476', 'License_Type': 'MD Full and Unrestricted'},
 {'License_Number': '2012-01719', 'License_Type': 'MD Full and Unrestricte

In [88]:
len(license_dict_list)

11954

In [52]:
index+=1

In [66]:
nc_lic[index:].sort_values('ME').to_csv('../../Data/License/ordered_lic.csv', index=False)

In [69]:
LIC_LIST = list(nc_lic[index:].sort_values('ME',ascending = False)['LIC_NBR'])

In [71]:
index

0

In [91]:
status_1 = pd.read_csv("../../Data/License/NC_License_Status_1.csv")
status_2 = pd.read_csv("../../Data/License/NC_License_Status_2.csv")

In [94]:
license_status_df = pd.concat([status_1, status_2]).drop_duplicates()

In [95]:
nc_lic

,ME,FIRST_NAME,LAST_NAME,LIC_NBR
0,102000030,MARK,JACKSON,200201481
1,102000129,CORY,BEAN,200201219
2,102000196,REBECCA,SUGG,2007-00835
3,102000200,CHARLES,BLAKE,2005-01659
4,102000242,JOSEPH,MOBLEY,2010-01708
...,...,...,...,...
74727,96501870037,SYED,HUSSAIN,9601638
74728,96501880059,NASEEM,ISMAIL,200200552
74729,96501890038,BETSY,PANICKER,2009-02099
74730,96501960044,MWIZA,GAUSI,2016-02379


In [103]:
NC_LIC_STATUS = pd.merge(license_status_df, nc_lic, left_on='License_Number', right_on='LIC_NBR', how='right')

In [102]:
nc_lic[nc_lic['LIC_NBR'].isin(NC_LIC_STATUS.LIC_NBR)==False]

,ME,FIRST_NAME,LAST_NAME,LIC_NBR
7328,1176080732,GABRIEL,JOHNSON,2018-00655


In [105]:
NC_LIC_STATUS.to_csv('NC_License_Status_2020-08-03.csv', index=False)

In [3]:
import pandas as pd
og = pd.read_excel("../../Data/License/NC_091020.xlsx")
df_1 = pd.read_csv("../../Data/License/NC_License_Status_2020-09-11.csv")
df_2 = pd.read_csv("../../Data/License/NC_License_Status_2020-09-12.csv")

In [4]:
len(og)

42703

In [8]:
df_all = pd.concat([df_1, df_2])

In [9]:
pd.merge(og, df_all, on = 'License_Number')

,License_Number,License_Type
0,2007-01286,Full and Unrestricted
1,2018-02560,Full and Unrestricted
2,2013-02286,Full and Unrestricted
3,2019-01910,Full and Unrestricted
4,2012-00426,Full and Unrestricted
...,...,...
25529,2009-01440,Full and Unrestricted
25530,2015-00720,Full and Unrestricted
25531,2015-01273,Full and Unrestricted
25532,2015-01831,Full and Unrestricted


In [12]:
df_all.drop_duplicates().to_excel('../../Data/NC_License_Status_2020-09-12.xlsx', index=False)

In [36]:
len(nc_lic)

42968

In [39]:
df_1 = pd.read_csv('../../Data/License/NC_License_Status_2020-11-11_1.csv')
df_2 = pd.read_csv('../../Data/License/NC_License_Status_2020-11-12_1.csv')
df_3 = pd.read_csv('../../Data/License/NC_License_Status_2020-11-12_2.csv')
df_4 = pd.read_csv('../../Data/License/NC_License_Status_2020-11-13_0.csv')
df_5 = pd.read_csv('../../Data/License/NC_License_Status_2020-11-13_1.csv')
df_6 = pd.read_csv('../../Data/License/NC_License_Status_2020-11-13_2.csv')
df_7 = pd.read_csv('../../Data/License/NC_License_Status_2020-11-13_3.csv')
df_8 = pd.read_csv('../../Data/License/NC_License_Status_2020-11-13_4.csv')

In [41]:
all_that = pd.concat([df_1,df_2,df_2,df_3,df_4,df_5,df_5,df_6, df_7,df_8])

In [43]:
all_that = all_that.drop_duplicates()

In [46]:
all_that = pd.merge(all_that, nc_lic, on='License_Number')

In [45]:
len(nc_lic)

42968

In [51]:
all_that = all_that.drop_duplicates('License_Number')

In [50]:
nc_lic[nc_lic.License_Number.isin(all_that.License_Number)==False]

,License_Number
39061,9800406


In [52]:
all_thst

'toria'

In [54]:
all_that.License_Type = [x[3:] for x in all_that.License_Type]

In [55]:
all_that

,License_Number,License_Type
0,2007-01286,Full and Unrestricted
1,2018-02560,Full and Unrestricted
2,2013-02286,Full and Unrestricted
3,38886,Full and Unrestricted
4,38647,Full and Unrestricted
...,...,...
42963,9501709,Full and Unrestricted
42964,24960,Full and Unrestricted
42965,2015-01273,Full and Unrestricted
42966,2015-01831,Full and Unrestricted


In [57]:
df2 = pd.DataFrame([['9800406', 'Full and Unrestricted']], columns=['License_Number','License_Type'])
all_that = all_that.append(df2)

In [58]:
nc_lic

,License_Number
0,2007-01286
1,2018-02560
2,2013-02286
3,38886
4,38647
...,...
42963,9501709
42964,24960
42965,2015-01273
42966,2015-01831


In [60]:
all_that = pd.merge(nc_lic, all_that, on='License_Number')

In [62]:
all_that[all_that['License_Type']=='Not found']

,License_Number,License_Type


In [63]:
all_that.to_excel('../../Data/License/NC_License_Status_2020-11-13.xlsx', index=False)